In [2]:
from nebula_execute import DB_execute
from nebula_pre_post import R3Post_Batch, R3_pre_process
from wayne_utils import load_data, save_data, len_compare
from tqdm import tqdm
import os
split_dict = load_data("/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/data/N3/split_dict.json", "json")

def split_merge( split_query ):
    order_dict = {}
    total_len = 0
    
    for key in ['disease', 'potter', 'nba']:
        dict_index_list = split_dict['index'][key]
        query_list = split_query[key]
        total_len += len(query_list)
        for i in range(len(query_list)):
            index = dict_index_list[i]
            if index not in order_dict:
                order_dict[index] = (query_list[i], key)
            else:
                raise Exception( f"Index重复:{index}")
    if total_len != len(order_dict):
        raise Exception( f"长度不一致:{total_len, len(order_dict)}")
    ret_list = []
    space_list = []
    for i in range( total_len ):
        query, key = order_dict[i ]
        space_list.append( key)
        ret_list.append( query)
    return ret_list, space_list

space_list_dict = {}
for key in split_dict['index'].keys():
    lists = split_dict['index'][key]
    for i in range(len(lists)):
        index = lists[i]
        space_list_dict[index] = key
space_list = [ space_list_dict[i] for i in range(len(space_list_dict))]

## 执行

In [3]:
# for model_name in model_list:
# from_path_ = f"/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/inter_data/N3/query_list/FT-{model_name}_query_list.json"
# to_path = f"/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/inter_data/N3/execute_result_list/FT-{model_name}_execute_results.pickle"
model_name = "My"
from_path_ = "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/inter_data/N3/query_list/FT-GLM4-9B_query_list.json"
to_path = "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/inter_data/N3/execute_result_list/FT-GLM4-9B_execute_results.pickle"
query_list = load_data( from_path_, "json")
if isinstance( query_list, dict):
    query_list_all, space_list = split_merge( query_list )
else:
    query_list_all = query_list
query_list_all = R3_pre_process( query_list_all )
save_data( query_list_all, from_path_)
result_list = []
for i in tqdm( range( len(query_list_all)), desc=f"Processing {model_name}"):
    query = query_list_all[i]
    space_name = space_list[i] if space_list[i] != "potter" else "harrypotter_new"
    executor = DB_execute( space_name )
    result = executor.exec( query )
    result_list.append( result )
result_list_post = R3Post_Batch( result_list )
save_data( result_list_post, to_path)

Processing My:   0%|          | 0/1254 [00:00<?, ?it/s]

R3 postprocessing...: 100%|██████████| 1254/1254 [00:00<00:00, 561809.15it/s]
